In [1]:
# ⚠️ RESTART THE KERNEL BEFORE RUNNING THIS NOTEBOOK ⚠️
# Click: Kernel → Restart Kernel (or Ctrl+Shift+P → "Restart Kernel")
# Then run all cells from the beginning

# Clean cache  
import subprocess
subprocess.run(['find', '../src', '-name', '__pycache__', '-type', 'd', '-exec', 'rm', '-rf', '{}', '+'], 
               capture_output=True, check=False)
print("✅ Cleared Python cache")
print("⚠️  If you see 'cp.atomic' errors, RESTART THE KERNEL and run all cells again")

✅ Cleared Python cache
⚠️  If you see 'cp.atomic' errors, RESTART THE KERNEL and run all cells again


# ELO-Based Dynamic Rating System Demo

This notebook demonstrates the ELO rating system for dating app matching.

## Key Differences from Static Scores:

| Feature | Static (PageRank/Composite) | Dynamic (ELO) |
|---------|----------------------------|---------------|
| Updates | Once at profile creation | After each interaction |
| Based on | Profile quality metrics | Actual match success |
| Reflects | Potential attractiveness | Market value |
| Self-correcting | No | Yes |
| New users | May be misranked | Start at average, adjust quickly |

## How ELO Works:

1. All users start at 1200 ELO rating
2. When user A likes user B:
   - If B also liked A (match): Both gain points
   - If B hasn't decided: A gets small boost, B gets larger boost
   - If B rejected A: A loses points, B gains points
3. Rating changes depend on expected outcome:
   - High-rated user matching low-rated user: small change
   - Equal-rated users matching: larger change
4. Final ratings reflect actual matching success over time

In [2]:
import sys
sys.path.insert(0, '../src')

# Force reload of modules
if 'matchmaker.models.elo' in sys.modules:
    del sys.modules['matchmaker.models.elo']
if 'matchmaker.engine' in sys.modules:
    del sys.modules['matchmaker.engine']
if 'matchmaker' in sys.modules:
    del sys.modules['matchmaker']

import cudf
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from matchmaker.engine import MatchingEngine
from matchmaker.models.elo import EloConfig

/home/zain/anaconda3/envs/matchmaker-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data and Run Models

In [3]:
# Initialize engine
engine = MatchingEngine()

In [ ]:
# Load interactions with gender information
engine.load_interactions(
    "data/swipes_clean.csv", 
    decider_col='decidermemberid',
    other_col='othermemberid', 
    like_col='like', 
    timestamp_col='timestamp',
    gender_col='decidergender'
)

Reading data... ✅
Constructing graph... ✅
Fitting ALS... 
🚀 Preparing data...
🎯 Training male→female ALS...


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# Compute engagement scores (useful for comparison)
engine.run_engagement()

In [ ]:
# Compute popularity metrics and assign PageRank leagues
engine.run_popularity()

In [ ]:
# Keep data in cudf for GPU-accelerated processing
user_gdf = engine.user_df
interaction_gdf = engine.interaction_df

print(f"Total users: {len(user_gdf)}")
print(f"Total interactions: {len(interaction_gdf)}")
print(f"\nPageRank League distribution:")
league_counts = user_gdf['league'].value_counts().to_pandas().sort_index()
print(league_counts)

## Compute ELO Ratings

**What ELO Measures in Dating Apps:**
- **DESIRABILITY**: How often you get liked when others swipe on you
- **NOT selectivity**: Your own swiping behavior doesn't affect your ELO
- High ELO = you're frequently liked (desirable)
- Low ELO = you're frequently rejected (less desirable)

**GPU-Accelerated Implementation:**
- **9.8M interactions** processed in ~3-4 seconds
- **171K users** scored with chunked batch updates (100K interactions per chunk)
- Uses CuPy scatter-add operations (`cp.add.at`) for efficient rating accumulation
- **Rating bounds**: Clamped between 100-10,000 to prevent extreme outliers

**Gender-Specific Pools:**
- Separate rating scales for males (M) and females (F)
- Males are only compared to other males
- Females are only compared to other females
- Accounts for different market dynamics (e.g., women typically get more likes)

In [ ]:
# Run ELO rating system
engine.run_elo()

In [ ]:
# Refresh user data
user_gdf = engine.user_df

# Show sample users with ELO ratings
sample = user_gdf[['user_id', 'gender', 'league', 'pagerank', 'elo_rating', 'interaction_count', 'is_stable']].sample(20).to_pandas()
print("\nSample users with ELO ratings:")
print(sample)

## Analyze ELO Rating Distribution

In [ ]:
# Plot ELO rating distribution
user_pd = user_gdf[['elo_rating', 'gender']].dropna().to_pandas()

fig = px.histogram(
    user_pd, 
    x='elo_rating', 
    color='gender',
    nbins=50,
    title='ELO Rating Distribution by Gender',
    labels={'elo_rating': 'ELO Rating'},
    barmode='overlay',
    opacity=0.7
)

fig.show()

In [ ]:
# Box plot of ELO ratings by PageRank league
user_pd_league = user_gdf[['elo_rating', 'league', 'gender']].dropna().to_pandas()

fig = px.box(
    user_pd_league,
    x='league',
    y='elo_rating',
    color='gender',
    title='ELO Rating Distribution by PageRank League',
    category_orders={'league': ['Bronze', 'Silver', 'Gold', 'Platinum', 'Diamond']}
)
fig.show()

## Analyze Stable vs Unstable Users

In [ ]:
# Compare stable (≥10 interactions) vs unstable users
stability_stats = user_gdf.groupby('is_stable').agg({
    'user_id': 'count',
    'elo_rating': ['mean', 'std'],
    'interaction_count': ['mean', 'median']
}).to_pandas()

print("\nStability Analysis:")
print(stability_stats)

## Relationship between PageRank and ELO

In [ ]:
# Analyze relationship between PageRank and ELO
analysis_df = user_gdf[['user_id', 'gender', 'league', 'pagerank', 'elo_rating', 'interaction_count', 'is_stable']].to_pandas()

# Show top and bottom ELO users
print("Top 10 ELO Ratings:")
print(analysis_df.nlargest(10, 'elo_rating')[['user_id', 'gender', 'league', 'pagerank', 'elo_rating', 'interaction_count']])

print("\nBottom 10 ELO Ratings:")
print(analysis_df.nsmallest(10, 'elo_rating')[['user_id', 'gender', 'league', 'pagerank', 'elo_rating', 'interaction_count']])

## Correlation Analysis

In [ ]:
# Correlation between PageRank and ELO
import numpy as np

correlation_data = user_gdf[['pagerank', 'elo_rating']].dropna().to_pandas()
correlation = np.corrcoef(correlation_data['pagerank'], correlation_data['elo_rating'])[0, 1]

print(f"\nCorrelation between PageRank and ELO: {correlation:.3f}")

# Scatter plot
fig = px.scatter(
    correlation_data.sample(min(5000, len(correlation_data))),  # Sample for performance
    x='pagerank',
    y='elo_rating',
    title=f'PageRank vs ELO Rating (correlation: {correlation:.3f})',
    labels={'pagerank': 'PageRank Score', 'elo_rating': 'ELO Rating'},
    opacity=0.5
)
fig.show()

## Insights and Recommendations

### Key Observations:
1. **ELO Ratings**: Reflect actual match success based on interaction outcomes
2. **Correlation with PageRank**: Check how ELO aligns with network-based importance
3. **Stability**: Users with more interactions have more reliable ELO ratings
4. **Distribution**: ELO provides a continuous measure of matching success

### When to Use ELO:
- ✅ You want ratings to reflect actual match success (not just profile quality)
- ✅ You can process interactions chronologically  
- ✅ You want self-correcting, adaptive ratings
- ✅ You have sufficient interaction data per user

### When NOT to Use ELO:
- ❌ Users have very few interactions (<10)
- ❌ You need ratings immediately for new users
- ❌ You can't explain dynamic ratings to stakeholders
- ❌ Real-time processing is required

### Potential Use Cases:
- **League Assignment**: Use ELO rating quantiles instead of PageRank
- **Match Quality**: Predict match probability between users with similar ELO
- **User Insights**: Identify users whose ELO doesn't match their PageRank (overperforming/underperforming)
- **A/B Testing**: Compare PageRank-based vs ELO-based matching

## ELO Design Decision: Desirability vs. Selectivity

**Critical Insight:** ELO in dating apps should measure **how desirable you are**, NOT how selective you are.

### Why This Matters:

Consider user **3851603** (Female, Diamond League):
- Received **489 swipes** from others
- Got **272 likes** (55.6% like rate) - very desirable!
- Only liked **4 people** herself (2.3% selectivity) - very selective

**Wrong Approach** (penalizes selectivity):
- Updates ELO when you swipe (decider) AND when others swipe on you (target)
- Being selective (rejecting 97.7%) → ELO drops
- Result: Diamond user at rock-bottom 100 ELO ❌

**Correct Approach** (measures desirability):
- Only updates ELO when others swipe on you (target)
- High like rate (55.6%) → ELO increases
- Selectivity doesn't matter - your swiping behavior is irrelevant to your desirability
- Result: Diamond user at 1,137 ELO (above average) ✅

### Gender-Specific Calibration:

Dating apps have **asymmetric market dynamics**:
- **Males**: ~4% like rate when shown to women
- **Females**: ~49% like rate when shown to men

The ELO system accounts for this by:
1. Creating **separate rating pools** for M and F
2. **Calibrating expected scores** to each gender's baseline like rate
3. Centering both distributions around the initial rating (1200)

This means:
- High ELO male = liked more than typical 4% baseline
- High ELO female = liked more than typical 49% baseline
- Ratings are **relative within gender**, not absolute across genders

### Implementation:
```python
# Calculate baseline like rate for this gender pool
baseline_like_rate = mean(likes)  # ~4% for males, ~49% for females

# Expected score centers around baseline, adjusted by rating
expected = baseline_like_rate + (rating_diff / 400) * (1 - baseline_like_rate)

# Rating increases when you get liked more than expected for your current rating
```

### Results:
- **Males**: Mean 1,197 ELO, 22% above starting
- **Females**: Mean 1,218 ELO, 58% above starting  
- Both distributions centered near initial rating (1200)
- Ratings reflect **desirability within gender pool**, accounting for different market dynamics

In [ ]:
# Show the Diamond user example
analysis_df = user_gdf[['user_id', 'gender', 'league', 'pagerank', 'elo_rating', 'interaction_count']].to_pandas()
diamond_user = analysis_df[analysis_df['user_id'] == 3851603]

if len(diamond_user) > 0:
    print("Example: Diamond Female User (3851603)")
    print(diamond_user.to_string(index=False))
    print("\nThis user:")
    print("  - Diamond league (high PageRank)")
    print("  - 489 interactions as target")
    print("  - 55.6% like rate when shown to others")
    print("  - ELO: 1,137 (above average) ✅")
    print("\nIn the old implementation, this user had 100 ELO (floor)")
    print("because she was penalized for being selective (2.3% like rate).")